In [1]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from stop_words import get_stop_words
#from scipy import sparse
from scipy import spatial
import gensim
import utils
import sys
import string
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from utils import random_idx
from utils import utils
from utils import lang_vectors_utils as lvu
from utils import documents
%matplotlib inline

num_topics = 50
passes = 20
#topn = 10
k = 5000
N = 10000
# cluster_sizes is mapping to n-gram size
# cluster_sz in random_idx referring to specific element (int) in cluster_sizes, array
cluster_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
ordered = 1
#assuming this is the alphabet bc of precedent in generate_text.py
#alph = 'abc' 
alphabet = string.lowercase + ' '
RI_letters = random_idx.generate_letter_id_vectors(N, k, alphabet)
# words should be unique? (such as permutations)
# number of words added in a syntax vector
syntax_granularities = [1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
lower_n_cutoff = .85

height has been deprecated.

2016-09-16 13:07


In [26]:
def vectorize_documents(documents):
    vectors = {}
    for doc in documents:
        for word in doc:
            vectors[word] = random_idx.id_vector(N, word, alphabet, RI_letters)
    return vectors

def similarity_matrix(dictionary,id_to_word):
    """
    calculate cosine similarity of every word
    """
    num_words = len(id_to_word)
    sm = np.zeros((num_words,num_words))
    cutoff_index = int(lower_n_cutoff*num_words)
    cutoff = float("inf")

    for row in range(0,num_words):
        for col in range(0,num_words):
            sm[row][col] = np.dot(np.transpose(dictionary[id_to_word[row]][0]), dictionary[id_to_word[col]][0])
        # find the 85% smallest element in array
        cutoff = min(cutoff, np.partition(sm[row], cutoff_index)[cutoff_index])
    return sm, cutoff

def syntax_space(similarity_matrix, vectorized_dictionary, id_to_word):
    """
    number of granularities of words encoded per syntax vector x number of tokens x N
    index in numpy array on axis "number of tokens" equivalent to token id
    http://stackoverflow.com/questions/6910641/how-to-get-indices-of-n-maximum-values-in-a-numpy-array
    """
    num_tokens = similarity_matrix.shape[0]
    matrices = np.zeros((len(syntax_granularities), num_tokens, N))
    for gran_i in range(len(syntax_granularities)):
        for row in range(num_tokens):
            granularity = min(syntax_granularities[gran_i],num_tokens)
            most_similar_indices = np.argpartition(similarity_matrix[row],-granularity)[-granularity:]
            
            for key in most_similar_indices:
                matrices[gran_i][row] = np.add(matrices[gran_i][row],vectorized_dictionary[id_to_word[key]][0])
    return matrices

In [3]:
# run tests
def accuracy_syntax(test_files, tokenized_test_documents, dictionary, vectorized_dictionary, syntaxed_space, cutoff, syntax_granularities):
    token_to_id = dictionary.token2id
    accuracy_per_granularity = np.zeros((len(test_files),syntaxed_space.shape[0]))
    for i in range(len(test_files)):
        test_tokens = tokenized_test_documents[i]
        for gran_i in range(len(syntax_granularities)):
            for test_token in test_tokens:
                #number of granularities of words encoded per syntax vector x number of tokens x N
                #N, dictionary[i], alphabet, RI_letters
                prediction_vector = np.dot(np.transpose(random_idx.id_vector(N, test_token, alphabet, RI_letters)), syntaxed_space[gran_i])
                prediction_id = np.argmax(prediction_vector)
                # if test token is inside dictionary
                if test_token in token_to_id.keys():
                    if prediction_id == token_to_id[test_token]:
                        accuracy_per_granularity[i][gran_i]+=1
                # account for new tokens and similarity not identity
                # >= bottom 85% cutoff for same vector group from similarity matrix
                elif prediction_vector >= cutoff:
                    accuracy_per_granularity[i][gran_i]+=1
            accuracy_per_granularity[i] /= syntax_granularities[gran_i]
    return accuracy_per_granularity


In [4]:
def graphing_3d():
    # each vector maps to the test_token. dots graphs but mappings can printed
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    xs =[1,2,3,4,5,6,7,8,9,10]
    ys =[5,6,2,3,13,4,1,2,4,8]
    zs =[2,3,3,3,5,7,9,11,9,10]

    xt =[-1,-2,-3,-4,-5,-6,-7,8,-9,-10]
    yt =[-5,-6,-2,-3,-13,-4,-1,2,-4,-8]
    zt =[-2,-3,-3,-3,-5,-7,9,-11,-9,-10]

    ax.scatter(xs, ys, zs, c='r', marker='o')
    ax.scatter(xt, yt, zt, c='b', marker='^')

    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

    plt.show()

def graphing_2d(test_files, syntax_granularities, accuracy_per_granularity):
    for i in range(accuracy_per_granularity.shape[0]):
        print test_files[i]
        fig = plt.figure()
        ax = fig.add_subplot(11, projection='2d')
        ax.scatter(syntax_granularities, accuracy_per_granularity[i], c='r', marker='o')

        ax.set_xlabel('granularity')
        ax.set_ylabel('accuracy %')

        plt.show()

In [5]:
# create sample documents
raw_path = "raw_texts/texts_english/"
preprocessed_path = "preprocessed_texts/english/"
training_preprocessed_path = "preprocessed_texts/english/with_spaces/"

training_files = ["a_christmas_carol.txt", "alice_in_wonderland.txt"]
# this is for testing accuracy against the 
# actual stream that will be the test input
test_files = ["hamlet_english.txt", "percy_addleshaw.txt"]

In [6]:
training_doc_set = documents.create_doc_set(training_preprocessed_path, training_files)
test_doc_set = documents.create_doc_set(preprocessed_path, test_files)

training_documents = [doc.split() for doc in training_doc_set]
test_documents = [doc.split() for doc in test_doc_set]




In [21]:
vectorized_dictionary = vectorize_documents(training_documents)
id_to_word = {}
words = vectorized_dictionary.keys()
for i in range(len(words)):
    id_to_word[i] = words[i]

In [27]:
# similarity matrix
sm, syntax_cutoff = similarity_matrix(vectorized_dictionary, id_to_word)

In [29]:
syntaxed_space = syntax_space(sm, vectorized_dictionary, id_to_word) 

KeyboardInterrupt: 

In [ ]:
accuracy_per_granularity = accuracy_syntax(test_files, test_documents, dictionary, vectorized_dictionary, syntaxed_space, cutoff, syntax_granularities)

In [ ]:
graphing_2d(test_files, syntax_granularities, accuracy_per_granularity)